In [1]:
import lxml.etree as et
from lxml import etree
import numpy as np
import h5py, os, sys

try:
    import openmc
except ImportError:
    None

AVAGADRO = 6.02214076e23

In [2]:
# Additional Functions Relating to Materials_XML_Parser
def Atomic_Mass(Nuc):  # Temp Solution, simplifies to the Atomic Mass Number
    # Pulls Data from the NIST Nuclude Data file if it exists
    if os.path.exists("nist-nuclide-data.txt"):
        Symbol = ""
        Mass_Num = ""
        for _ in Nuc:
            try:
                int(_)
                Mass_Num += _
            except:
                Symbol += _
        Mass_Num = int(Mass_Num)
        NAT_AB = False
        if Symbol == "H":
            if Mass_Num == 2:
                Symbol = "D"
            elif Mass_Num == 3:
                Symbol = "T"
        if any([_ == Symbol for _ in ["C", "Zn", "Pt", "Os", "Tl"]]) and Mass_Num == 0:
            NAT_AB = True
        with open("nist-nuclide-data.txt", "r") as Data:
            Data_Lines = Data.readlines()[:]
            for n, line in enumerate(Data_Lines):
                if line[16:-1] == Symbol:
                    if int(Data_Lines[n + 1][14:-1]) == Mass_Num:
                        AM = float(Data_Lines[n + 2][23:-1].split("(")[0])
                        break
                    elif NAT_AB:
                        AM = float(Data_Lines[n + 4][25:-1].split("(")[0])
                        break
            Data_Lines.clear()
    # Runs OpenMC function if NIST data does not exist and OpenMC module is installed
    elif "openmc" in sys.modules:
        AM = openmc.atomic_mass(Nuc)
    # Approximates atomic mass to atomic mass number (int) if either other method fails
    else:
        AM = ""
        for _ in Nuc:
            try:
                AM += str(int(_))
            except:
                None
        AM = int(AM)
        print(
            f"WARNING: Using integer approximation for {Nuc}.\nEnsure nist-nuclide-data.txt or OpenMC are available."
        )
    return AM


def Average_Molar_Mass(Material=[], Percent=[], Percent_Type=[]):
    # Fuction pulled from OpenMC, calculates avg molar mass based on nuclide composition
    mass = 0
    moles = 0
    assert len(Material) == len(Percent_Type) == len(Percent)
    for i in range(len(Material)):
        if Percent_Type[i] == "ao":
            mass += Percent[i] * Atomic_Mass(Material[i])
            moles += Percent[i]
        elif Percent_Type[i] == "wo":
            mass += Percent[i]
            moles += Percent[i] / Atomic_Mass(Material[i])
    return mass / moles


def Reconstruct_Scatter_Matrix(Data=dict()):
    # Method only works if Scatter Data is saved properly by OpenMC cross sections file
    Flat_Scatter = np.array(
        Data["scatter_data"]["scatter_matrix"]
    )  # Pulls Mx1 scatter data
    g_min = (
        np.array(Data["scatter_data"]["g_min"]) - 1
    )  # Determines min group number of rows
    g_max = (
        np.array(Data["scatter_data"]["g_max"]) - 1
    )  # Determines max group number of rows
    Num_Groups = len(g_min)
    Scatter_Matrix = np.zeros([Num_Groups, Num_Groups])
    for i in range(Num_Groups):
        N = g_max[i] - g_min[i] + 1  # Defines N values to pull from Mx1 scatter data
        Scatter_Matrix[i, g_min[i] : g_max[i] + 1] = Flat_Scatter[
            :N
        ]  # Fills GxG matrix from min to max of row i
        Flat_Scatter = np.delete(Flat_Scatter, range(N))  # Removes used data
    Scatter_Matrix = np.transpose(
        Scatter_Matrix
    )  # Proper formatting for Scatter Matrix
    return Scatter_Matrix

In [3]:
# Additional Functions Relating to Geometry_XML_Parser
def Region_Fill(py_file=None, Cell=dict()):
    if py_file == None:
        py_file = open("_.py", "t+a")
    Regions = Cell.get("region")
    if Regions != None:
        Region_IDs = np.array([val for val in Regions.split()])
        # py_file.write("region=") # Temporarily removed in case of redundancy
    else:
        return  # Exits function if cell has no regions

    Skip = True
    for i in range(len(Region_IDs)):
        Region_ID = Region_IDs[i]
        # Runs through CSG operators, skips '&' for '|', continues if left blank
        if not Skip and Region_ID[0] != "|":
            py_file.write(" & ")
        if Region_ID[0] == "~":
            py_file.write("~")
            Region_ID = Region_ID[1:]
        elif Region_ID[0] == "|":
            py_file.write(" | ")
            Region_ID = Region_ID[1:]
        Skip = False
        if Region_ID == "":
            Skip = True
            continue
        # Looks for parathesis goruping and ensures only ID int remains
        while type(Region_ID) == str or type(Region_ID) == np.str_:
            if Region_ID[0] == "(":
                py_file.write("(")
                Region_ID = Region_ID[1:]
            if Region_ID[-1] == ")":
                close_join += 1
                Region_ID = Region_ID[:-1]
            else:
                close_join = 0
            try:
                Region_ID = int(Region_ID)
            except:
                None
        # Applys + or - half-spaces, then writes surface into region
        if Region_ID > 0:
            r_sign = "+"
        elif Region_ID < 0:
            r_sign = "-"
        py_file.write(f"{r_sign}s{int(abs(Region_ID))}")
        py_file.write(close_join * ")")  # Closes grouped and subgrouped regions
    py_file.write(", ")
    if py_file.name == "_.py":
        py_file.close()


def Write_Universes(
    Geom_Root,
    py_file,
    Lat_ID=None,
    Complex=False,
    Exclude_List=None,
    Defined_Cells=None,
):
    global created_universes

    Univ_Data = np.array([[0, 0]])
    Written_Universes = []
    for Cell in Geom_Root.findall("cell"):
        if (
            Cell.get("material") != None and Lat_ID == None and not Complex
        ):  # Pulls material universes
            Univ_ID = int(Cell.get("universe"))
            Cell_ID = int(Cell.get("id"))
            Univ_Data = np.append(Univ_Data, np.array([[Univ_ID, Cell_ID]]), axis=0)
        elif (
            Cell.get("fill") != None and Lat_ID == None and Complex
        ):  # Pulls complex universes that are not excluded
            if int(Cell.get("fill")) in Exclude_List:
                continue
            Univ_ID = int(Cell.get("universe"))
            Cell_ID = int(Cell.get("id"))
            Univ_Data = np.append(Univ_Data, np.array([[Univ_ID, Cell_ID]]), axis=0)
        elif Cell.get("fill") == str(Lat_ID):  # Pulls lattice univere with specified ID
            Univ_ID = int(Cell.get("universe"))
            Cell_ID = int(Cell.get("id"))
            Univ_Data = np.append(Univ_Data, np.array([[Univ_ID, Cell_ID]]), axis=0)
    if Univ_Data.shape != (1, 2):
        # Constructs Univsere dictionary grouping cell IDs to Universe IDs
        Univ_Data = np.delete(Univ_Data, 0, 0).T
        Univ_Dict = {
            n: rep[n]
            for rep in [{}]
            for i, n in enumerate(Univ_Data[0, :])
            if rep.setdefault(n, []).append(i) or len(rep[n]) == 1
        }
        for key in Univ_Dict:
            if Complex:
                if not all(
                    [
                        (_ in Defined_Cells)
                        for _ in [Univ_Data[1, i] for i in Univ_Dict[key]]
                    ]
                ):
                    continue

            if (
                key not in created_universes
            ):  # If u# is not already defined, it creates u# with initial cells
                py_file.write(f"u{key} = mcdc.universe([")
                created_universes.append(key)
            else:  # If u# was already defined, uses u#.add_cells with new cells
                py_file.write(f"u{key}.add_cells([")
            for j, i in enumerate(Univ_Dict[key]):
                py_file.write(f"c{Univ_Data[1,i]}")
                if i != Univ_Dict[key][-1]:
                    py_file.write(", ")
                if j % 20 == 19:
                    py_file.write(
                        "\n\t\t"
                    )  # Moves to next line if there are more than 20 cells/line, compacts formatting
            if (key == 0) and (key not in created_universes):
                py_file.write(
                    "], root=True])\n"
                )  # If OpenMC uses universe 0 as root, sets u0 as the root universe
            else:
                py_file.write("])\n")
            Written_Universes.append(int(key))

    return Written_Universes


def Write_Rect_Lattice(Lattice, py_file, Lat_ID=None):
    # Creates lattice configuration for 1D, 2D, or 3D rectangular lattices, Hexagonal lattices not implemented yet.
    # Identifies individual lattices based on Lat_ID
    py_file.write(f"\n# Rectangular Lattice {Lat_ID}\n")
    if Lattice.get("name") != None:
        py_file.write(f"# Name: {Lattice.get('name')}\n")
    py_file.write(f"Lattice{Lat_ID} = mcdc.lattice(")
    dims = np.array([int(val) for val in Lattice.find("dimension").text.split()])
    pitch = np.array([float(val) for val in Lattice.find("pitch").text.split()])
    Lower_Left = np.array(
        [float(val) for val in Lattice.find("lower_left").text.split()]
    )
    xyz_str = ["x", "y", "z"]
    for i in range(len(pitch)):
        py_file.write(
            "{}=[{}, {}, {}], ".format(xyz_str[i], Lower_Left[i], pitch[i], dims[i])
        )
    # Pulls lattice str from xml and reformats into desired shape based on dimensions
    Lattice_xml = np.array(Lattice.find("universes").text.split("\n"))
    Lattice_xml = np.delete(Lattice_xml, np.where(Lattice_xml == ""))
    Lattice_xml = np.array([ind.split() for ind in Lattice_xml]).flat
    Lattice_xml = np.delete(Lattice_xml, np.where(Lattice_xml == "")).astype(int)
    Lattice_Fin = np.reshape(Lattice_xml, dims[::-1])

    # Fills in universes, determines positions for [ and ] and seperates into lines matching shape
    py_file.write(f"universes=" + (len(dims) - 1) * "[")
    if len(dims) == 2:
        dims = np.append(dims, 1)[::-1]
        Lattice_Fin = np.reshape(Lattice_Fin, dims)
    else:
        dims = dims[::-1]
    for i in range(dims[0]):
        for j in range(dims[1]):
            py_file.write("\n\t[")
            for k in range(dims[2]):
                py_file.write("u{}".format(Lattice_Fin[i, j, k]))
                if k != dims[2] - 1:
                    py_file.write(", ")
            py_file.write("]")
            if j != dims[1] - 1:
                py_file.write(",")
        if i != dims[0] - 1:
            py_file.write("],[")
    py_file.write((len(pitch) - 1) * "]" + ")\n")


def Convert_Setting_Value(Sett_Val=None):
    # Tests if Sett_Val is bool arg as string
    if Sett_Val == "true":
        Sett_Val = True
    elif Sett_Val == "false":
        Sett_Val = False
    # Tests and corrects if Sett_Val is an int or float value
    else:
        try:
            Sett_Val = float(Sett_Val)
        except:
            None  # Val is str
        try:
            Sett_Val = int(Sett_Val)
        except:
            None  # Val is str or float
    # Otherwise returns str as it was initially input
    return Sett_Val


In [4]:

# XML Parser/Translator Functions
def Materials_XML_Parser(XML_file="materials.xml", py_file=None):
    """Sub-parcer for materials

    .. version:: 0.1.5

    """

    if py_file == None:
        py_file = open("Materials_Only.py", "t+w")
        py_file.write("import mcdc\nimport numpy as np\n\n")
    else:
        py_file.write(
            "\n" + 30 * "#" + "\n#__________Materials__________\n" + 30 * "#" + "\n\n"
        )
    Mats_Parse = et.parse(XML_file)
    Mats_Root = Mats_Parse.getroot()
    if Mats_Root.tag == "model":
        Mats_Root = Mats_Root.find("materials")

    if Mats_Root.find("cross_sections") != None:
        MXS_File = h5py.File(Mats_Root.find("cross_sections").text, "r")

        # Create MC/DC MGXS
        file_name = "mcdc-" + Mats_Root.find("cross_sections").text
        with h5py.File(file_name, "w") as f:
            for material in list(MXS_File.keys()):
                for temperature in list(MXS_File[material].keys()):
                    if temperature == "kTs":
                        continue

                    name = material + "-" + temperature + "/"
                    data = MXS_File[material][temperature]

                    total = data["total"][:]
                    G = len(total)

                    fission = np.zeros(G)
                    nu = np.zeros(G)
                    chi = np.zeros((G, G))

                    if "fission" in list(data.keys()):
                        fission = data["fission"][:]
                        nu_fission = data["nu-fission"][:]
                        nu = nu_fission / fission

                        chi_1d = data["chi"][:]
                        for g in range(G):
                            chi[g, :] = chi_1d[g]

                    absorption = data["absorption"][:]
                    capture = absorption - fission
                    scatter = Reconstruct_Scatter_Matrix(data)

                    f.create_dataset(name + "capture", data=capture)
                    f.create_dataset(name + "fission", data=fission)
                    f.create_dataset(name + "scatter", data=scatter)
                    f.create_dataset(name + "nu", data=nu)
                    f.create_dataset(name + "chi", data=chi)

        py_file.write(
            f"import h5py\nMXS_Lib = h5py.File('mcdc-{Mats_Root.find('cross_sections').text}')\n"
        )

    for Mat in Mats_Root.findall("material"):
        Mat_ID = int(Mat.get("id"))
        Mat_name = Mat.get("name")
        if Mat_name != None:
            py_file.write(f"# Material Name: {Mat_name}\n")
        if Mat.get("temperature") != None:
            py_file.write(f"# Temperature: {Mat.get('temperature')}K\n")
        if Mat.find("sab") != None:
            for Sab in Mat.findall("sab"):
                Sab_name = Sab.get("name")
                py_file.write(f"# S(a,b): {Sab_name} (Not Implemented)\n")
        if Mat.get("depletable") != None:
            py_file.write("# Depletable\n")

        density_units = Mat.find("density").get("units")
        sum_density = False
        MACRO_Pull = False
        if density_units != "sum" and density_units != "macro":
            density = float(Mat.find("density").get("value"))
            if density_units == "g/cm3" or density_units == "g/cc":
                density *= 1
                den_in_atom = False
            elif density_units == "kg/m3":
                density *= 0.001
                den_in_atom = False
            elif density_units == "atom/barn-cm":
                density *= 1
                den_in_atom = True
            elif density_units == "atom/cm3" or density_units == "atom/cc":
                density *= 1.0e-24
                den_in_atom = True
        elif density_units == "sum":
            sum_density = True
            den_in_atom = True
            density = 0
        elif density_units == "macro":
            MACRO_Pull = True

        if not MACRO_Pull:
            Nuc_Names = np.array([])
            Nuc_Den_Values = np.array([])
            Nuc_Den_Types = np.array([])
            for Nuc in Mat.findall("nuclide"):
                Nuc_Names = np.append(Nuc_Names, Nuc.get("name"))
                if Nuc.get("ao") != None:
                    Nuc_Den_Values = np.append(Nuc_Den_Values, float(Nuc.get("ao")))
                    Nuc_Den_Types = np.append(Nuc_Den_Types, "ao")
                elif Nuc.get("wo") != None:
                    Nuc_Den_Values = np.append(Nuc_Den_Values, float(Nuc.get("wo")))
                    Nuc_Den_Types = np.append(Nuc_Den_Types, "wo")
                else:
                    print("No type")
            AVG_Molar_Mass = Average_Molar_Mass(
                Nuc_Names, Nuc_Den_Values, Nuc_Den_Types
            )
            percent_in_atom = np.all(Nuc_Den_Types == "ao")

            if sum_density:
                density = np.sum(Nuc_Den_Values)
            if not percent_in_atom:
                for n, nuc in enumerate(Nuc_Names):
                    Nuc_Den_Values[n] *= AVG_Molar_Mass / Atomic_Mass(nuc)
            sum_percent = np.sum(Nuc_Den_Values)
            Nuc_Den_Values /= sum_percent
            if not den_in_atom:
                density *= AVAGADRO / AVG_Molar_Mass * 1e-24
            Nuc_Den_Values *= density

            py_file.write(f"m{Mat_ID} = mcdc.material(")
            py_file.write("nuclides=[\n")
            for i in range(len(Nuc_Names)):
                if Nuc_Names[i] == "U234" or Nuc_Names[i] == "Th232" or Nuc_Names[i] == "Pu241" or Nuc_Names[i] == "U236":
                    continue
                if Nuc_Names[i] == "C0":
                    Nuc_Names[i] == "C12"
                py_file.write(f"\t['{Nuc_Names[i]}',{Nuc_Den_Values[i]}]")
                if Nuc_Names[i] != Nuc_Names[-1]:
                    py_file.write(",\n")
                else:
                    py_file.write("])\n")

        elif MACRO_Pull:
            py_file.write(f"# Cross-sections From: mcdc-{MXS_File.filename}\n")
            Mat_Key = Mat.find("macroscopic").get("name")
            OpenMC_SubKey = list(MXS_File[Mat_Key].keys())[0]
            py_file.write(f"M{Mat_ID}_Lib = MXS_Lib['{Mat_Key}-{OpenMC_SubKey}']\n")
            py_file.write(f"m{Mat_ID} = mcdc.material(\n")
            py_file.write(f"capture = M{Mat_ID}_Lib['capture'],\n")
            py_file.write(f"scatter = M{Mat_ID}_Lib['scatter'],\n")
            py_file.write(f"fission = M{Mat_ID}_Lib['fission'],\n")
            py_file.write(f"nu_p = M{Mat_ID}_Lib['nu'],\n")
            py_file.write(f"chi_p = M{Mat_ID}_Lib['chi'],\n")
            py_file.write(f")\n")

    py_file.write(f"mvoid = mcdc.material(nuclides=[['N14',1e-10]])")
    if Mats_Root.find("cross_sections") != None:
        MXS_File.close()
        py_file.write("MXS_Lib.close()\n")

    if py_file.name == "Materials_Only.py":
        py_file.close()
        print(f"Materials MCDC File: ./{py_file.name}")
    return

In [5]:
created_universes = []

In [6]:

def Geometry_XML_Parser(XML_file="geometry.xml", py_file=None):
    """Subparcer for geometry

    .. version:: 0.1.4

    """

    if py_file == None:
        py_file = open("Geometry_Only.py", "t+w")
        py_file.write("import mcdc\n\n")
    else:
        py_file.write(
            "\n" + 30 * "#" + "\n#__________Geometry__________\n" + 30 * "#" + "\n\n"
        )
    Geom_Parse = et.parse(XML_file)
    Geom_Root = Geom_Parse.getroot()
    if Geom_Root.tag == "model":
        Geom_Root = Geom_Root.find("geometry")

    py_file.write("# Surface(s)\n")
    for Surf in Geom_Root.findall("surface"):
        Surf_ID = int(Surf.get("id"))
        Surf_type = Surf.get("type")
        Surf_Coeffs = [float(val) for val in Surf.get("coeffs").split()]
        py_file.write("s{} = mcdc.surface(".format(Surf_ID))

        # PLANE GEOMETRY
        if Surf_type == "x-plane":
            Surf_type = "plane-x"
            py_file.write("'{}', x={f[0]}, ".format(Surf_type, f=Surf_Coeffs))
        elif Surf_type == "y-plane":
            Surf_type = "plane-y"
            py_file.write("'{}', y={f[0]}, ".format(Surf_type, f=Surf_Coeffs))
        elif Surf_type == "z-plane":
            Surf_type = "plane-z"
            py_file.write("'{}', z={f[0]}, ".format(Surf_type, f=Surf_Coeffs))
        elif Surf_type == "plane":
            py_file.write(
                "'{}', A={f[0]}, B={f[1]}, C={f[2]}, D={f[3]}, ".format(
                    Surf_type, f=Surf_Coeffs
                )
            )
        # CYLINDER GEOMETRY
        elif Surf_type == "x-cylinder":
            Surf_type = "cylinder-x"
            py_file.write(
                "'{}', center=[{f[0]}, {f[1]}], radius={f[2]}, ".format(
                    Surf_type, f=Surf_Coeffs
                )
            )
        elif Surf_type == "y-cylinder":
            Surf_type = "cylinder-y"
            py_file.write(
                "'{}', center=[{f[0]}, {f[1]}], radius={f[2]}, ".format(
                    Surf_type, f=Surf_Coeffs
                )
            )
        elif Surf_type == "z-cylinder":
            Surf_type = "cylinder-z"
            py_file.write(
                "'{}', center=[{f[0]}, {f[1]}], radius={f[2]}, ".format(
                    Surf_type, f=Surf_Coeffs
                )
            )
        # SPHERE GEOMETRY
        elif Surf_type == "sphere":
            py_file.write(
                "'{}', center=[{f[0]}, {f[1]}, {f[2]}], radius={f[3]}, ".format(
                    Surf_type, f=Surf_Coeffs
                )
            )
        # QUADRIC DEFINED GEOMETRY
        elif Surf_type == "quadric":
            py_file.write(
                "'{}', A={f[0]}, B={f[1]}, C={f[2]}, D={f[3]}, E={f[4]}, F={f[5]}, G={f[6]}, H={f[7]}, I={f[8]}, J={f[9]}, ".format(
                    Surf_type, f=Surf_Coeffs
                )
            )
        elif Surf_type == "x-cone":
            Surf_type = "quadric"
            R2 = -Surf_Coeffs[3]
            G = -2 * R2 * Surf_Coeffs[0]
            H = -2 * Surf_Coeffs[1]
            I = -2 * Surf_Coeffs[2]
            J = R2 * Surf_Coeffs[0] ** 2 + Surf_Coeffs[1] ** 2 + Surf_Coeffs[2] ** 2
            py_file.write(
                f"'{Surf_type}', A={R2}, B=1, C=1, G={G}, H={H}, I={I}, J={J}, "
            )
        elif Surf_type == "y-cone":
            Surf_type = "quadric"
            R2 = -Surf_Coeffs[3]
            G = -2 * Surf_Coeffs[0]
            H = -2 * R2 * Surf_Coeffs[1]
            I = -2 * Surf_Coeffs[2]
            J = Surf_Coeffs[0] ** 2 + R2 * Surf_Coeffs[1] ** 2 + Surf_Coeffs[2] ** 2
            py_file.write(
                f"'{Surf_type}', A=1, B={R2}, C=1, G={G}, H={H}, I={I}, J={J}, "
            )
        elif Surf_type == "z-cone":
            Surf_type = "quadric"
            R2 = -Surf_Coeffs[3]
            H = -2 * Surf_Coeffs[0]
            I = -2 * Surf_Coeffs[1]
            G = -2 * R2 * Surf_Coeffs[2]
            J = Surf_Coeffs[0] ** 2 + Surf_Coeffs[1] ** 2 + R2 * Surf_Coeffs[2] ** 2
            py_file.write(
                f"'{Surf_type}', A=1, B=1, C={R2}, G={G}, H={H}, I={I}, J={J}, "
            )
        # Note: General Cone in OpenMC is saved as a quadric
        # NOT TRANSLATABLE GEOMETRY (Torus)
        else:
            py_file.write(f")# {Surf_type} not supported")

        Surf_Bound = Surf.get("boundary")
        if Surf_Bound == None or Surf_Bound == "transmission":
            bc = "interface"
        elif Surf_Bound == "reflective" or Surf_Bound == "white":
            bc = "reflective"
        elif Surf_Bound == "vacuum":
            bc = "vacuum"
        else:
            print("Boundary condition not accepted, defaulted to interface")
            bc = "interface"
        py_file.write(f"bc='{bc}')")

        if Surf.get("name") != None:
            py_file.write(f" # Name: {Surf.get('name')}\n")
        else:
            py_file.write("\n")

    Written_Cells = []
    py_file.write("\n# Material Cell(s)\n")
    for Cell in Geom_Root.findall("cell"):
        if Cell.get("material") == None:
            continue
        Cell_ID = int(Cell.get("id"))
        Written_Cells.append(Cell_ID)

        py_file.write("c{} = mcdc.cell(".format(Cell_ID))
        Region_Fill(py_file=py_file, Cell=Cell)
        py_file.write(f"fill=")

        Mat_IDs = np.array([val for val in Cell.get("material").split()])
        for i in range(len(Mat_IDs)):
            py_file.write(f"m{Mat_IDs[i]}")
            if Mat_IDs[i] != Mat_IDs[-1]:
                py_file.write(", ")
        py_file.write(")")

        if Cell.get("name") != None:
            py_file.write(f" # Name: {Cell.get('name')}\n")
        else:
            py_file.write(f"\n")

    py_file.write("# Root Universe Cells List:\nu0_cells = []\n")
    py_file.write("# Material Universe(s)\n")
    # FIX: this line needs to be present for universes
    # Written_Universes = Write_Universes(Geom_Root, py_file)

    Lat_IDs_List = []
    for Lattice in Geom_Root.findall("lattice"):
        Lat_IDs_List.append(int(Lattice.get("id")))

    Exclude_List = Lat_IDs_List.copy()
    Writing_Complex = True
    Level = 1

    while Writing_Complex:
        if len(Geom_Root.findall("cell")) == (len(Written_Cells) + len(Lat_IDs_List)):
            Writing_Complex = False
            continue
        py_file.write(f"\n# Complex Cell(s) Level {Level}\n")
        for Cell in Geom_Root.findall("cell"):
            if Cell.get("fill") == None:
                continue
            Fill_ID = int(Cell.get("fill"))
            if not (Fill_ID in Exclude_List) and (Fill_ID in Written_Universes):
                Cell_ID = int(Cell.get("id"))
                Written_Cells.append(Cell_ID)

                py_file.write("c{} = mcdc.cell(".format(Cell_ID))

                Region_Fill(py_file=py_file, Cell=Cell)
                py_file.write(f"fill=u{Fill_ID}")

                if Cell.get("translation") is not None:
                    tr = Cell.get("translation").split()
                    py_file.write(f", translation=[{tr[0]}, {tr[1]}, {tr[2]}]")

                if Cell.get("rotation") is not None:
                    rot = Cell.get("rotation").split()[0]
                    py_file.write(f", rotation=[{rot[0]}, {rot[1]}, {rot[2]}]")

                py_file.write(")")

                if Cell.get("name") != None:
                    py_file.write(f" # Name: {Cell.get('name')}\n")
                else:
                    py_file.write(f"\n")
        py_file.write(f"# Complex Universe(s) Level {Level}\n")
        Complex_Universes = Write_Universes(
            Geom_Root,
            py_file,
            Complex=True,
            Exclude_List=Exclude_List,
            Defined_Cells=Written_Cells,
        )
        Exclude_List = Lat_IDs_List + Written_Universes
        Written_Universes += Complex_Universes
        if len(Complex_Universes) == 0:
            Writing_Complex = False
        else:
            Level += 1

    for Lattice in Geom_Root.findall(
        "lattice"
    ):  # Update so that if universe is redefefined, it is instead updated
        Lat_ID = Lattice.get("id")
        Write_Rect_Lattice(Lattice, py_file, Lat_ID)
        for Cell in Geom_Root.findall("cell"):
            if Cell.get("fill") == Lat_ID:
                Cell_ID = int(Cell.get("id"))

                py_file.write("c{} = mcdc.cell(".format(Cell_ID))
                Region_Fill(py_file=py_file, Cell=Cell)
                py_file.write(f"fill=Lattice{Lat_ID}")

                if Cell.get("translation") is not None:
                    tr = Cell.get("translation").split()
                    py_file.write(f", translation=[{tr[0]}, {tr[1]}, {tr[2]}]")

                if Cell.get("rotation") is not None:
                    rot = Cell.get("rotation").split()[0]
                    py_file.write(f", rotation=[{rot[0]}, {rot[1]}, {rot[2]}]")

                py_file.write(")")

                if Cell.get("name") != None:
                    py_file.write(f" # Name: {Cell.get('name')}\n")
                else:
                    py_file.write(f"\n")
        Lattice_Universe = Write_Universes(Geom_Root, py_file, Lat_ID)
        Written_Universes += Lattice_Universe

    if py_file.name == "Geometry_Only.py":
        py_file.close()
        print(f"Geometry MCDC File: ./{py_file.name}")
    return

In [7]:

def Settings_XML_Parser(XML_file="settings.xml", py_file=None):
    if py_file == None:
        py_file = open("Settings_Only.py", "t+w")
        py_file.write("import mcdc\nimport numpy as np\n\n")
    else:
        py_file.write(
            "\n" + 30 * "#" + "\n#__________Settings__________\n" + 30 * "#" + "\n\n"
        )

    Sett_Parse = et.parse(XML_file)

    Sett_Root = Sett_Parse.getroot()
    if Sett_Root.tag == "model":
        Sett_Root = Sett_Root.find("settings")

    Settings_Dict = {}

    for Sett_key in [_.tag for _ in Sett_Root]:
        if isinstance(Sett_key,str) == 0:
            continue
        if Sett_key in ["particles", "batches", "inactive", "source", "output"]:
            continue  # Ignores listed common settings
        elif Sett_key in [
            "cutoff",
            "keff_trigger",
            "mesh",
            "resonance_scattering",
            "state_point",
            "source_point",
            "surf_source_read",
            "surf_source_write",
            "tabular_legendre",
            "trigger",
            "volume_calc",
            "weight_windows",
            "weight_window_generator",
            "weight_window_checkpoints",
        ]:  # Settings with Sub-Dictionaries
            Sett_Val = {}
            Parent = Sett_Root.find(Sett_key)
            if Parent is not None:
                for Sub in Parent:
                    Sub_key = Sub.tag
                    Sub_Val = Sub.text
                    Sett_Val[Sub_key] = Convert_Setting_Value(Sub_Val)
        else:  # All other uncommon settings with simple values
            Element = Sett_Root.find(Sett_key)
            if Element is not None and Element.text is not None:
                Sett_Val = Element.text
                Sett_Val = Convert_Setting_Value(Sett_Val)
            else:
                Sett_Val = None  # Or another default value if needed  
        Settings_Dict[Sett_key] = Sett_Val

    if len(Settings_Dict) != 1:
        py_file.write("# Original Conditions (Some Not Implemented)\n")
    for (
        key
    ) in (
        Settings_Dict.keys()
    ):  # Writes uncommon settings that are not implemented (Note: Weighting and Mesh not yet implemented)
        if not key in ["run_mode", "seed"]:
            py_file.write(f"#\t{key} = {Settings_Dict[key]}\n")

    py_file.write("# Simulation Parameters\n")
    # Possible modes: eigenvalue, fixed source, plot, volume, particle reset (Last three not implemented)
    if Settings_Dict["run_mode"] == "eigenvalue":
        N_Particles = int(Sett_Root.find("particles").text)
        N_Batches = int(Sett_Root.find("batches").text)
        N_Inactive = int(Sett_Root.find("inactive").text)
        N_Active = N_Batches - N_Inactive
        # FIX: I manually set these to a particle count I wanted for the icsbep benchmarks. Change to actually copy openmc
        #py_file.write(f"mcdc.eigenmode(N_inactive={N_Inactive}, N_active={N_Active})\n")
        py_file.write(f"mcdc.eigenmode(N_inactive=20, N_active=180)\n")
        #py_file.write(f"mcdc.setting(N_particle={N_Particles}")
        py_file.write(f"mcdc.setting(N_particle=10000")
    elif Settings_Dict["run_mode"] == "fixed source":
        N_Particles = int(Sett_Root.find("particles").text)
        N_Batches = int(Sett_Root.find("batches").text)
        py_file.write(f"mcdc.setting(N_particle={N_Particles}, N_batch={N_Batches}")
    else:
        py_file.write(f"# Run mode: {Settings_Dict['run_mode']} Not Supported")

    # Uses same seed int if recorded and closes settings function
    if "seed" in Settings_Dict.keys():
        py_file.write(f", rng_seed={Settings_Dict['seed']})\n")
    else:
        py_file.write(")\n")

    py_file.write("\n# Source Parameters\n")
    Source = Sett_Root.find("source")
    if Source == None:
        py_file.write(
            "# Particle: neutron\n# Space Type: Point\n mcdc.source(point=[0.0,0.0,0.0], prob=1.0)\n"
        )
    else:
        Particle = Source.get("particle")
        if Particle == None:
            Particle = "neutron"
        # P_Type = Source.get('type')
        openmc_source_strength = Source.get("strength")
        if openmc_source_strength == None:
            Strength = 1.0
        else:
            Strength = float(Source.get("strength"))
        Space = Source.find("space")
        #Space_Type = Space.get("type")
        if Space.find("type") is not None:
            Space_Type = Space.find("type").text 
        else:
            Space_Type = "point"
        


        py_file.write(f"# Particle: {Particle}\n")
        py_file.write(f"# Space Type: {Space_Type}")

        if Space_Type == "point":
            if Space.find("type") is not None:
                Parameters = [
                    float(val) for val in Space.find("parameters").text.split()
                ]
            else:
                Parameters = [0.0, 0.0, 0.0]
            py_file.write(
                "\nmcdc.source(point=[{P[0]},{P[1]},{P[2]}], prob={S})\n".format(
                    P=Parameters, S=Strength
                )
            )
        elif Space_Type == "cartesian":
            Parameters = [0, 0, 0]
            py_file.write(" (Not Implemented, Replaced with Point)\n")
            py_file.write(
                "mcdc.source(point=[{P[0]},{P[1]},{P[2]}], prob={S})\n".format(
                    P=Parameters, S=Strength
                )
            )
        elif Space_Type == "box" or Space_Type == "fission":
            Parameters = [
                #float(val) for val in Space.find("parameters").text.split(" ")
                float(val) for val in Space.find("parameters").text.split() if val.strip()
            ]
            py_file.write(
                "\nmcdc.source(x=[{P[0]},{P[3]}], y=[{P[1]},{P[4]}], z=[{P[2]},{P[5]}], prob={S})\n".format(
                    P=Parameters, S=Strength
                )
            )
        elif Space_Type == "spherical":
            Parameters = [float(val) for val in Space.get("origin").split()]
            Parameters += [
                float(val) for val in Space.find("r").get("parameters").split()
            ]
            Parameters += [
                float(val)
                for val in Space.find("cos_theta").get("parameters").split()
            ]
            Parameters += [
                float(val) for val in Space.find("phi").get("parameters").split()
            ]
            py_file.write(" (Not Implemented, Replaced with Point)\n")
            py_file.write(
                "mcdc.source(point=[{P[0]},{P[1]},{P[2]}], prob={S})\n".format(
                    P=Parameters, S=Strength
                )
            )
        elif Space_Type == "cylindrical":
            Parameters = [0, 0, 0]
            py_file.write(" (Not Implemented, Replaced with Point)\n")
            py_file.write(
                "mcdc.source(point=[{P[0]},{P[1]},{P[2]}], prob={S})\n".format(
                    P=Parameters, S=Strength
                )
            )
        elif Space_Type == "mesh":
            Parameters = [0, 0, 0]
            py_file.write(" (Not Implemented, Replaced with Point)\n")
            py_file.write(
                "nmcdc.source(point=[{P[0]},{P[1]},{P[2]}], prob={S})\n".format(
                    P=Parameters, S=Strength
                )
            )

    if py_file.name == "Settings_Only.py":
        py_file.close()
        print(f"Settings MCDC File: ./{py_file.name}")
    return


In [8]:

# Ultimate function for running all parsers
def OpenMC_to_MCDC(
    output_name="input.py",
    materials_xml="materials.xml",
    geometry_xml="geometry.xml",
    settings_xml="settings.xml",
    model_xml=None,
):
    """Constructs python file (output_name) into folder location. Defines nuclear system input deck compatable with MC/DC
    terminology from OpenMC input deck files (materials.xml, geometry.xml, settings.xml, etc.) within folder location.

    .. version:: 0.1.4

    Parameters
    ----------
    output_name : str, optional
        Name of new python file, must end in '.py'
    materials_xml : str, optional
        Name of materials file, must end in '.xml'
    geometry_xml : str, optional
        Name of geometry file, must end in '.xml'
    settings_xml : str, optional
        Name of settings file, must end in '.xml'
    model_xml : str, optional
        Name of model file, must end in '.xml',
        overrides individual xml files

    Returns
    -------
    None\n
    Prints created file link
    """

    # Check input parameters
    assert output_name[-3:] == ".py", "New file must be a python file"
    assert materials_xml[-4:] == ".xml", "Improper Geometry input file name"
    assert geometry_xml[-4:] == ".xml", "Improper Materials input file name"
    assert settings_xml[-4:] == ".xml", "Improper Settings input file name"
    if model_xml is not None and os.path.isfile(model_xml):
        assert model_xml[-4:] == ".xml", "Improper Model input file name"
        materials_xml = model_xml
        geometry_xml = model_xml
        settings_xml = model_xml

    # Create file
    MCDC_Code = open(output_name, "t+w")

    # Imports
    MCDC_Code.write("import mcdc\nimport numpy as np\n\n")

    # Parse materials, geometry, and settings
    Materials_XML_Parser(XML_file=materials_xml, py_file=MCDC_Code)
    Geometry_XML_Parser(XML_file=geometry_xml, py_file=MCDC_Code)
    Settings_XML_Parser(XML_file=settings_xml, py_file=MCDC_Code)

    MCDC_Code.write("\nmcdc.run()\n\n")

    # Close file and report
    MCDC_Code.close()
    print("MCDC Input File Constructed: .\{}".format(output_name))


In [9]:


def find_and_translate(base_dir):
    """
    Recursively find all openmc directories containing the required XML files, 
    create mcdc directory next to them, and run the translator.
    """
    required_files = {"materials.xml", "geometry.xml", "settings.xml"}
    num_failed = 0
    num_built = 0
    failed_list = []
    for root, dirs, files in os.walk(base_dir):
        #if root != "/usr/workspace/northroj/dane_mcdc_openmc/icsbep/openmc/heu-met-fast-002/openmc/case-5":
        #    continue
        try:
            # Check for case 1: XMLs directly in openmc folder
            if os.path.basename(root) == "openmc" and required_files.issubset(set(files)):
                openmc_dir = root
                parent_dir = os.path.dirname(openmc_dir)
                problem_name = os.path.basename(parent_dir)
                mcdc_dir = os.path.join(parent_dir, "mcdc")
                
                # Create mcdc directory if it doesn't exist
                if not os.path.exists(mcdc_dir):
                    os.makedirs(mcdc_dir)

                # Name the output file using the problem name
                output_name = os.path.join(mcdc_dir, f"{problem_name}.py")
                materials_xml = os.path.join(openmc_dir, "materials.xml")
                geometry_xml = os.path.join(openmc_dir, "geometry.xml")
                settings_xml = os.path.join(openmc_dir, "settings.xml")
                
                # Run the translator
                print(f"Translating OpenMC inputs in {openmc_dir} to {output_name}...")
                OpenMC_to_MCDC(
                    output_name=output_name,
                    materials_xml=materials_xml,
                    geometry_xml=geometry_xml,
                    settings_xml=settings_xml,
                )
                print(f"Translation complete: {output_name}")
                num_built += 1

            # Check for case 2: XMLs in subfolders of openmc (e.g., openmc/<casename>/)
            elif os.path.basename(os.path.dirname(root)) == "openmc" and required_files.issubset(set(files)):
                case_dir = root
                openmc_dir = os.path.dirname(case_dir)
                parent_dir = os.path.dirname(openmc_dir)
                problem_name = os.path.basename(parent_dir)
                case_name = os.path.basename(case_dir)
                mcdc_dir = os.path.join(parent_dir, "mcdc")
                
                # Create mcdc directory if it doesn't exist
                if not os.path.exists(mcdc_dir):
                    os.makedirs(mcdc_dir)

                # Name the output file using problem name and case name
                output_name = os.path.join(mcdc_dir, f"{problem_name}_{case_name}.py")
                materials_xml = os.path.join(case_dir, "materials.xml")
                geometry_xml = os.path.join(case_dir, "geometry.xml")
                settings_xml = os.path.join(case_dir, "settings.xml")
                
                # Run the translator
                print(f"Translating OpenMC inputs in {case_dir} to {output_name}...")
                OpenMC_to_MCDC(
                    output_name=output_name,
                    materials_xml=materials_xml,
                    geometry_xml=geometry_xml,
                    settings_xml=settings_xml,
                )
                print(f"Translation complete: {output_name}")
                num_built += 1

        except Exception as e:
            print(f"Exeption: {e}")
            num_failed += 1
            failed_list.append(openmc_dir)
        
    print(num_failed, "failures")
    print(num_built, "built")
    print(failed_list)

In [1]:
# Specify the base directory to start the search
base_dir = "<path_to_MCDC-verification>/MCDC-verification/icsbep/openmc"
find_and_translate(base_dir)

NameError: name 'find_and_translate' is not defined